# Week 18: Cumulative Trading Strategy (Weeks 1-18)

## Integration: Foundation through Advanced Portfolio Optimization

**New in Week 18:** Mean-Variance, Black-Litterman, Risk Parity, HRP

| Week Range | Topics Integrated |
|------------|------------------|
| 1-4 | Foundation, Statistics, Time Series, ML Basics |
| 5-8 | Portfolio Optimization, Linear/Factor Models, Trees, Volatility |
| 9-12 | Unsupervised Learning, Forecasting, Feature Engineering, Backtesting |
| 13-17 | Neural Networks, RNN/LSTM, Transformers, RL, Options/Hedging |
| **18** | **Mean-Variance, Black-Litterman, Risk Parity, HRP** |

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import squareform
import warnings
warnings.filterwarnings('ignore')

TRADING_DAYS = 252
RISK_FREE_RATE = 0.05
np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Libraries loaded (Weeks 1-18)")

In [ ]:
# Download multi-asset data
tickers = ['SPY', 'QQQ', 'IWM', 'EFA', 'EEM', 'GLD', 'TLT', 'IEF', 'VNQ', 'DBC']
data = yf.download(tickers, start='2018-01-01', end='2024-01-01', progress=False, auto_adjust=True)
prices = data['Close'].dropna()
returns = prices.pct_change().dropna()
print(f"✅ Data: {len(prices)} days, {len(tickers)} assets")

## Week 18: Portfolio Optimization Methods

In [ ]:
class PortfolioOptimizer:
    """Multiple portfolio optimization methods (Week 18)."""
    
    def __init__(self, returns):
        self.returns = returns
        self.n_assets = returns.shape[1]
        self.mean_returns = returns.mean() * TRADING_DAYS
        self.cov_matrix = returns.cov() * TRADING_DAYS
        
    def mean_variance(self, target='sharpe'):
        """Mean-Variance Optimization (Week 5)."""
        def neg_sharpe(w):
            ret = np.dot(w, self.mean_returns)
            vol = np.sqrt(np.dot(w.T, np.dot(self.cov_matrix, w)))
            return -(ret - RISK_FREE_RATE) / vol
        
        def portfolio_vol(w):
            return np.sqrt(np.dot(w.T, np.dot(self.cov_matrix, w)))
        
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        init = np.ones(self.n_assets) / self.n_assets
        
        obj = neg_sharpe if target == 'sharpe' else portfolio_vol
        result = minimize(obj, init, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x
    
    def risk_parity(self):
        """Risk Parity (Week 18)."""
        def risk_budget_obj(w):
            vol = np.sqrt(np.dot(w.T, np.dot(self.cov_matrix, w)))
            mrc = np.dot(self.cov_matrix, w) / vol
            rc = w * mrc
            target_rc = vol / self.n_assets
            return np.sum((rc - target_rc)**2)
        
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
        bounds = tuple((0.01, 1) for _ in range(self.n_assets))
        init = np.ones(self.n_assets) / self.n_assets
        
        result = minimize(risk_budget_obj, init, method='SLSQP', bounds=bounds, constraints=constraints)
        return result.x
    
    def black_litterman(self, views, view_confidences):
        """Black-Litterman (Week 18)."""
        tau = 0.05
        delta = 2.5  # Risk aversion
        
        # Market equilibrium weights (cap-weighted proxy)
        mkt_weights = np.ones(self.n_assets) / self.n_assets
        
        # Implied returns
        pi = delta * np.dot(self.cov_matrix, mkt_weights)
        
        # Views matrix P and Q
        P = np.array(views['matrix'])
        Q = np.array(views['returns'])
        omega = np.diag(view_confidences)
        
        # Black-Litterman formula
        tau_sigma = tau * self.cov_matrix
        M = np.linalg.inv(np.linalg.inv(tau_sigma) + P.T @ np.linalg.inv(omega) @ P)
        bl_returns = M @ (np.linalg.inv(tau_sigma) @ pi + P.T @ np.linalg.inv(omega) @ Q)
        
        # Optimize with BL returns
        self.mean_returns = bl_returns
        return self.mean_variance(target='sharpe')
    
    def hierarchical_risk_parity(self):
        """Hierarchical Risk Parity - HRP (Week 18)."""
        # Correlation-based distance
        corr = self.returns.corr()
        dist = np.sqrt((1 - corr) / 2)
        
        # Hierarchical clustering
        dist_condensed = squareform(dist.values)
        link = linkage(dist_condensed, method='single')
        sorted_idx = leaves_list(link)
        
        # Recursive bisection
        def get_cluster_var(cov, items):
            cov_slice = cov.iloc[items, items]
            ivp = 1 / np.diag(cov_slice)
            ivp /= ivp.sum()
            return np.dot(ivp, np.dot(cov_slice, ivp))
        
        def get_recursive_bisection(cov, sorted_idx):
            weights = pd.Series(1.0, index=sorted_idx)
            clusters = [sorted_idx]
            
            while len(clusters) > 0:
                clusters = [c[j:k] for c in clusters for j, k in 
                           ((0, len(c)//2), (len(c)//2, len(c))) if len(c) > 1]
                for i in range(0, len(clusters), 2):
                    c0, c1 = clusters[i], clusters[i+1]
                    var0 = get_cluster_var(cov, c0)
                    var1 = get_cluster_var(cov, c1)
                    alpha = 1 - var0 / (var0 + var1)
                    weights[c0] *= alpha
                    weights[c1] *= 1 - alpha
            return weights
        
        cov = self.returns.cov() * TRADING_DAYS
        hrp_weights = get_recursive_bisection(cov, sorted_idx.tolist())
        return hrp_weights.sort_index().values

# Initialize optimizer
optimizer = PortfolioOptimizer(returns)

# Calculate all portfolios
portfolios = {
    'Equal Weight': np.ones(len(tickers)) / len(tickers),
    'Max Sharpe': optimizer.mean_variance('sharpe'),
    'Min Variance': optimizer.mean_variance('minvol'),
    'Risk Parity': optimizer.risk_parity(),
    'HRP': optimizer.hierarchical_risk_parity()
}

print("\n💼 PORTFOLIO WEIGHTS (Week 18)")
print("="*80)
weights_df = pd.DataFrame(portfolios, index=tickers)
print(weights_df.round(3).to_string())

In [ ]:
# Backtest all portfolios
print("\n📊 PORTFOLIO PERFORMANCE COMPARISON")
print("="*80)

results = []
for name, weights in portfolios.items():
    port_returns = (returns * weights).sum(axis=1)
    ann_ret = port_returns.mean() * TRADING_DAYS
    ann_vol = port_returns.std() * np.sqrt(TRADING_DAYS)
    sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol
    
    cum = (1 + port_returns).cumprod()
    max_dd = ((cum - cum.expanding().max()) / cum.expanding().max()).min()
    
    results.append({
        'Portfolio': name,
        'Return': ann_ret,
        'Volatility': ann_vol,
        'Sharpe': sharpe,
        'Max DD': max_dd
    })

results_df = pd.DataFrame(results).set_index('Portfolio')
print(results_df.round(4).to_string())

## Summary: Weeks 1-18 Integration

| Week | Concept | Implementation |
|------|---------|---------------|
| 1-2 | Data & Statistics | Multi-asset returns, correlations |
| 5 | Basic Portfolio | Mean-Variance Optimization |
| **18** | **Advanced Portfolio** | **Risk Parity, Black-Litterman, HRP** |

✅ Complete portfolio optimization framework with multiple methods!